In [224]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import PIL
import copy
import csv
from tqdm import tqdm
from skimage import io, color, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
from torch import nn
from sklearn.metrics import f1_score

In [225]:
import torch
import nibabel as nib
import os
import torch.nn.functional as F
from tqdm.auto import tqdm
import numpy as np

In [249]:
import torch
import nibabel as nib
import os
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import numpy as np
import time
from tqdm.auto import tqdm

def make_img_path(cid):
    case_id = "case_{:05d}".format(cid)
    root=os.path.join('./data',case_id)
    return os.path.join(root,'imaging.nii.gz')

def make_seg_path(cid):
    case_id = "case_{:05d}".format(cid)
    root=os.path.join('./data',case_id)
    return os.path.join(root,'segmentation.nii.gz')

if not os.path.exists('./data_npy'):
    os.mkdir('./data_npy')
if not os.path.exists('./data_npy/train'):
    os.mkdir('./data_npy/train')
if not os.path.exists('./data_npy/valid'):
    os.mkdir('./data_npy/valid')

count = 0
root = os.path.join('.', 'data_npy', 'train', 'image')
if not os.path.exists(root):
    os.mkdir(root)

for i in tqdm(range(160)):
    img_path = make_img_path(i)
    if 'case_00005' in img_path: #escludo file corrotti
        
        continue

    print("Processing file:", img_path)  # Aggiunto per stampare il percorso del file
    img = nib.load(make_img_path(i)).get_fdata()
    img = (img - img.min()) / (max((img.max() - img.min()), 1e-3))

    for j in range(len(img)):
        path = os.path.join(root, '{:05d}'.format(j + count))
        np.save(path, img[j:j + 1].astype(np.float32))

    count += len(img)




count=0
root=os.path.join('.','data_npy','train','segmentation')
if not os.path.exists(root):
    os.mkdir(root)

root_segmentation = os.path.join('.', 'data_npy', 'train', 'segmentation', 'kidney' )
if not os.path.exists(root_segmentation):
    os.mkdir(root_segmentation)

for i in tqdm(range(160)):
    seg = nib.load(make_seg_path(i)).get_fdata()
    seg_labels = np.zeros_like(seg, dtype=np.uint8)
    seg_labels[seg > 0] = 1  # Imposta a 1 i reni
    seg_labels[seg > 1] = 2  # Imposta a 2 il tumore

    for j in range(len(seg_labels)):
        path_segmentation = os.path.join(root_segmentation, 'kidney_{:05d}'.format(j + count))
        seg_1ch = torch.tensor(seg_labels[j:j + 1], dtype=torch.int64)
        seg_2ch = F.one_hot(seg_1ch, num_classes=3)
        seg_2ch = torch.squeeze(seg_2ch.permute(3, 0, 1, 2))
        seg_2ch = np.array(seg_2ch, dtype=np.uint8)
        try:
            np.save(path_segmentation, seg_2ch)
        except Exception as e:
            print("Error saving file:", path_segmentation)
            print("Error message:", str(e))
        count += 1
        
print("creato cartella segmentation")

  0%|          | 0/160 [00:00<?, ?it/s]

Processing file: ./data/case_00000/imaging.nii.gz


  1%|          | 1/160 [00:13<35:26, 13.37s/it]

Processing file: ./data/case_00001/imaging.nii.gz


  1%|▏         | 2/160 [00:26<34:13, 13.00s/it]

...

Processing file: ./data/case_00158/imaging.nii.gz


 99%|█████████▉| 159/160 [12:15<00:11, 11.94s/it]

Processing file: ./data/case_00159/imaging.nii.gz


100%|██████████| 160/160 [33:29<00:00, 12.56s/it]

dovrebbe aver creato cartella segmentation


In [250]:
count=0
root=os.path.join('.','data_npy','valid','image')
if not os.path.exists(root):
    os.mkdir(root)
    
for i in tqdm(range(161,210)):
    img=nib.load(make_img_path(i)).get_fdata()
    img=(img-img.min())/(max((img.max()-img.min()),1e-3))
    
    for j in range(len(img)):
        path=os.path.join(root,'{:05d}'.format(j+count))
        np.save(path,img[j:j+1].astype(np.float32))
    count+=len(img)

count=0
root=os.path.join('.','data_npy','valid','segmentation')
if not os.path.exists(root):
    os.mkdir(root)
    
for i in tqdm(range(161,210)):
    eg = nib.load(make_seg_path(i)).get_fdata()
    seg_labels = np.zeros_like(seg, dtype=np.uint8)
    seg_labels[seg > 0] = 1  # Imposta a 1 i reni
    seg_labels[seg > 1] = 2  # Imposta a 2 il tumore

    for j in range(len(seg_labels)):
        path_segmentation = os.path.join(root_segmentation, 'kidney_{:05d}'.format(j + count))
        seg_1ch = torch.tensor(seg_labels[j:j + 1], dtype=torch.int64)
        seg_2ch = F.one_hot(seg_1ch, num_classes=3)
        seg_2ch = torch.squeeze(seg_2ch.permute(3, 0, 1, 2))
        seg_2ch = np.array(seg_2ch, dtype=np.uint8)
        try:
            np.save(path_segmentation, seg_2ch)
        except Exception as e:
            print("Error saving file:", path_segmentation)
            print("Error message:", str(e))
        count += 1

100%|██████████| 49/49 [31:37<00:00, 38.73s/it]


In [251]:
import torch
import os
from torch.utils.data import Dataset
import numpy as np
import random
from torchvision import transforms
class KitsDataset(Dataset):
    def __init__(self, kind, output_classes, location='data_npy', tuning=False, tune_size=0.1, transform=None, normalize=False):
        self.kind = kind
        self.root = os.path.join(location, kind)
        self.img_path = os.path.join(self.root, 'image')
        self.seg_path = os.path.join(self.root, 'segmentation', 'kidney')
        self.tuning = tuning
        self.total = 37250 if self.kind == 'train' else 7922
        self.tune_list = []
        self.tune_cnt = 0
        self.transform = transform
        self.normalization = normalize
        self.output_classes = output_classes
        if self.normalization:
            self.normalize = transforms.Normalize((0.5,), (0.5,))

        if self.tuning:
            self.tune_cnt = int(self.total * tune_size)
            self.tune_list = np.random.choice(self.total, self.tune_cnt, replace=False)

    def __len__(self):
        num = self.tune_cnt if self.tuning else self.total
        return num

    def __getitem__(self, idx):
        idx = self.tune_list[idx] if self.tuning else idx
        num_channels = 3
        img_id = "{:05d}.npy".format(idx)
        final_img_path = os.path.join(self.img_path, img_id.format(idx))
        final_seg_path = os.path.join(self.seg_path, "kidney_{:05d}.npy".format(idx)) if self.kind == 'train' else os.path.join(self.seg_path, "kidney_{:05d}.npy".format(idx))

        if idx > 36415 or not os.path.exists(final_img_path) or not os.path.exists(final_seg_path):
    # Il file non esiste o è al di là del range desiderato, gestisci il caso appropriato (ad esempio, passa al successivo)
           return None


        img = torch.tensor(np.load(final_img_path), dtype=torch.float32)
        kidney_seg = torch.tensor(np.load(final_seg_path), dtype=torch.float32)


        img = torch.tensor(np.load(final_img_path), dtype=torch.float32)
 
       # Se l'immagine ha un solo canale, espandila per avere tre canali
        if img.dim() == 2:
         img = img.unsqueeze(0).repeat(3, 1, 1)

        # Assicurati che l'immagine abbia esattamente tre canali
        if img.shape[0] == 1:
            img = img.repeat(3, 1, 1)

        # Restituisci solo i primi tre canali se l'immagine ha più di tre canali
        if img.shape[0] > 3:
           img = img[:3, :, :]

        # Controlla se la maschera ha solo una dimensione di canale
        if kidney_seg.dim() == 2:
         # Aggiungi una dimensione di canale
           kidney_seg = kidney_seg.unsqueeze(0)

         # Controlla se la maschera ha meno di 3 canali
        if kidney_seg.shape[0] < 3:
            # Ripeti i canali per ottenere 3 canali
               kidney_seg = kidney_seg.repeat(3, 1, 1)

         #  Controlla se la maschera ha più di 3 canali
        if kidney_seg.shape[0] > 3:
            # Riduci i canali a 3
            kidney_seg = kidney_seg[:3, :, :]

        #questo  perchè la NN si aspetta un input a 3
       
        if self.normalization:
            img = self.normalize(img)

        if self.normalization:
    # Assicurati che la normalizzazione sia applicata a ciascun canale separatamente
         for channel in range(img.shape[1]):
           img[:, channel] = self.normalize(img[:, channel])


        if self.transform is not None:
            seed = np.random.randint(2147483647)
            random.seed(seed)
            torch.manual_seed(seed)
            img = self.transform(img)

            random.seed(seed)
            torch.manual_seed(seed)
            kidney_seg = self.transform(kidney_seg)


        sample = {'image': img, 'kidney_mask': kidney_seg}

        return sample


In [252]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [203]:

def createModel(output_classes):
    my_model = models.segmentation.fcn_resnet101(pretrained=True)
    set_parameter_requires_grad(my_model, feature_extracting= True)
    my_model.classifier[4] = nn.Conv2d(512, output_classes, kernel_size=(1, 1), stride=(1, 1))
    my_model.aux_classifier[4] = nn.Conv2d(256, output_classes, kernel_size=(1, 1), stride=(1, 1))
    my_model.train() 
    return my_model

In [183]:
from torch.utils.data.dataloader import default_collate

def collate_fn(batch):
    batch = [sample for sample in batch if sample is not None]  # Rimuovi i campioni None
    return default_collate(batch)

In [209]:
import itertools

In [253]:
def train_model(model, criterion, trainloader, validloader, optimizer, metrics, bpath, num_epochs):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    # Usa la GPU se disponibile
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Logger
    fieldnames = ['epoch', 'Train_loss', 'Val_loss'] + \
        [f'Train_{m}' for m in metrics.keys()] + \
        [f'Val_{m}' for m in metrics.keys()]
    with open(os.path.join(bpath, 'log.csv'), 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

    for epoch in range(1, num_epochs+1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        batchsummary = {a: [0] for a in fieldnames}

        for phase in ['Train', 'Val']:
            if phase == 'Train':
                model.train()  # Imposta il modello in modalità di addestramento
                dataloaders = trainloader  # Seleziona il dataset per l'addestramento
            else:
                model.eval()   # Imposta il modello in modalità di valutazione
                dataloaders = validloader  # Seleziona il dataset per la validazione

            # Itera sui dati
            for sample in tqdm(iter(dataloaders)):
                if sample is None:
                 continue
                
                inputs = sample['image'].to(device)
                kidney_masks = sample['kidney_mask'].to(device)

                # Azzera i gradienti dei parametri
                optimizer.zero_grad()

                # Tieni traccia della storia solo durante l'addestramento
                with torch.set_grad_enabled(phase == 'Train'):
                    outputs = model(inputs)
                    loss = criterion(outputs['out'], kidney_masks)
                    y_pred = outputs['out'].data.cpu().numpy().ravel()
                    y_true = kidney_masks.data.cpu().numpy().ravel()
                    for name, metric in metrics.items():
                        if name == 'f1_score':
                            batchsummary[f'{phase}_{name}'].append(
                                metric(y_true > 0, y_pred > 0, average='weighted'))

                    # Esegui il backward e l'ottimizzazione solo durante l'addestramento
                    if phase == 'Train':
                        loss.backward()
                        optimizer.step()
            batchsummary['epoch'] = epoch
            epoch_loss = loss
            batchsummary[f'{phase}_loss'] = epoch_loss.item()
            print('{} Loss: {:.4f}'.format(
                phase, loss))
        for field in fieldnames[3:]:
            batchsummary[field] = np.mean(batchsummary[field])
        print(batchsummary)
        with open(os.path.join(bpath, 'log.csv'), 'a', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(batchsummary)
            # Effettua una copia profonda del modello
            if phase == 'Val' and loss < best_loss:
                best_loss = loss
                best_model_wts = copy.deepcopy(model.state_dict())

    # Carica i pesi del miglior modello
    model.load_state_dict(best_model_wts)
    return model
    

In [254]:
def image_viewer(output_classes, image, mask):
    
    x=mask[0].numpy()
    x*=output_classes # The function need the classes to be integer 
    io.imshow(color.label2rgb(x, image[0].numpy(), bg_label=0)) # Set bkg transparent and shows only
                                                                #  other classes on top of the input png     
plt.show()

In [255]:

def main():
    output_classes = 3  # Numero di classi nel tuo problema di classificazione

    # Crea i dataLoader
    trainds = KitsDataset(kind='train', location='data_npy', output_classes = output_classes)
    validds = KitsDataset(kind='valid', location='data_npy', output_classes = output_classes)

    ##for i in range(len(trainds)):
     #   sample = trainds[i]
#
 #       print(i, sample['image'].shape, sample['kidney_mask'].shape)

 #       if i == (len(trainds)%5)-1:
  #          plt.show()
   #         break 

    trainloader = DataLoader(trainds, batch_size=4, shuffle=True, collate_fn=collate_fn)
    validloader = DataLoader(validds, batch_size=2, shuffle=False, collate_fn=collate_fn)

    # Creazione del modello, del criterio, dell'ottimizzatore e delle metriche
    my_model = createModel(output_classes)
    criterion = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)
    metrics = {'f1_score': f1_score}
    bpath = '/Volumes/LOLLO_HDD1/BCV/cheat'
    
    # Addestramento del modello
    my_model_trained = train_model(my_model, criterion, trainloader, validloader, optimizer, metrics, bpath, num_epochs=30 )

    # Ottieni il primo batch dei dati di addestramento per eseguire il modello e valutarne le prestazioni
    for i_batch, sample_batched in enumerate(trainloader):
        print(i_batch, sample_batched['image'].size(),
              sample_batched['kidney_mask'].size())
        break



In [256]:
main()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FCN_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=FCN_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5
----------


  0%|          | 2/9313 [00:54<70:35:22, 27.29s/it]


KeyboardInterrupt: 